# How to Read in Multiple Volume Scans Into a Single Object

## Imports

In [ ]:
import xradar as xd
import xarray as xr
import numpy as np
import act
import glob
import pyart
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from open_radar_data import DATASETS

In [ ]:
radar_files = [
    "gucxprecipradarcmacppiS2.c1.20220314.021559.nc",
    "gucxprecipradarcmacppiS2.c1.20220314.024239.nc",
    "gucxprecipradarcmacppiS2.c1.20220314.025840.nc",
]
files = [DATASETS.fetch(file) for file in radar_files]

## Access the Data

In [ ]:
ds = xr.open_dataset(files[0], engine="cfradial1", group="sweep_0")

In [ ]:
def fix_angle(ds):
    ds["time"] = ds.time.load()  # Convert time from dask to numpy

    start_ang = 0  # Set consistent start/end values
    stop_ang = 360

    # Find the median angle resolution
    angle_res = ds.azimuth.diff("azimuth").median()

    # Determine whether the radar is spinning clockwise or counterclockwise
    median_diff = ds.azimuth.diff("time").median()
    ascending = median_diff > 0
    direction = 1 if ascending else -1

    # first find exact duplicates and remove
    ds = xd.util.remove_duplicate_rays(ds)

    # second reindex according to retrieved parameters
    ds = xd.util.reindex_angle(
        ds, start_ang, stop_ang, angle_res, direction, method="nearest"
    )

    ds = ds.expand_dims("volume_time")  # Expand for volumes for concatenation

    ds["volume_time"] = [np.nanmin(ds.time.values)]

    return ds

In [ ]:
# Concatenate in xarray ds
ds = xr.open_mfdataset(
    files,
    preprocess=fix_angle,
    engine="cfradial1",
    group="sweep_0",
    concat_dim="volume_time",
    combine="nested",
)

In [ ]:
ds = ds.xradar.georeference()

In [ ]:
ds.isel(volume_time=0).DBZ.plot(x="x", y="y", cmap="Spectral_r", vmin=-20, vmax=60)

In [ ]:
xd.georeference.get_crs(ds)
cart_crs = ccrs.Projection(proj_crs)